<a href="https://colab.research.google.com/github/ollayyy/Popularity-Predictor/blob/main/First_attempt_popularitypredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installs & Imports

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=a1e52e684ca410f2f71185ce7251beeb8dc83e966a3db6d440452fe9c3f0c13b
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
pip install findspark

In [3]:
import findspark
findspark.init()

In [4]:
import pandas as pd

In [5]:
import tensorflow as tf


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Call Data

In [9]:
spark = SparkSession.builder.appName("PopularityPredictor").getOrCreate()

In [10]:
csv_path = "/content/drive/MyDrive/Data Analysis Bootcamp/spotify_data.csv"
df = spark.read.csv(csv_path, header=True, quote= "\"", escape="\"")

In [11]:
df.show()

+---+--------------------+--------------------+--------------------+----------+----+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------------+
|_c0|         artist_name|          track_name|            track_id|popularity|year|   genre|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|duration_ms|time_signature|
+---+--------------------+--------------------+--------------------+----------+----+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------------+
|  0|          Jason Mraz|     I Won't Give Up|53QF56cjZA9RTuuMZ...|        68|2012|acoustic|       0.483| 0.303|  4| -10.058|   1|     0.0429|       0.694|             0.0|   0.115|  0.139|133.406|     240166|             3|
|  1|          Jason Mraz|    93 Million Miles|1s8tP3jP4GZcyHDsj...|        50|2012|acoustic|   

In [12]:
cols = ['popularity','year','genre','danceability','energy','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms']
cleaned_df = df.select(cols)
cleaned_df.show()

+----------+----+--------+------------+------+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+
|popularity|year|   genre|danceability|energy|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|duration_ms|
+----------+----+--------+------------+------+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+
|        68|2012|acoustic|       0.483| 0.303| -10.058|   1|     0.0429|       0.694|             0.0|   0.115|  0.139|133.406|     240166|
|        50|2012|acoustic|       0.572| 0.454| -10.286|   1|     0.0258|       0.477|        1.37e-05|  0.0974|  0.515|140.182|     216387|
|        57|2012|acoustic|       0.409| 0.234| -13.711|   1|     0.0323|       0.338|           5e-05|  0.0895|  0.145|139.832|     158960|
|        58|2012|acoustic|       0.392| 0.251|  -9.845|   1|     0.0363|       0.807|             0.0|  0.0797|  0.508|204.961|     304293|
|        54|2012|aco

In [13]:
clean_df=cleaned_df.toPandas()

In [14]:
df = pd.DataFrame(clean_df)
df = df.astype({"popularity":"int", "year":"int", "danceability":"float", "energy":"float", "loudness":"float", "mode":"int",
                "speechiness":"float", "acousticness":"float", "instrumentalness":"float", "liveness":"float", "valence":"float",
                "tempo":"float", "duration_ms":"int"})

Machine Learning

In [15]:
df.dtypes

popularity            int64
year                  int64
genre                object
danceability        float64
energy              float64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
duration_ms           int64
dtype: object

In [16]:
df=pd.get_dummies(df)

In [17]:
y=df['popularity'].values

In [18]:
x=df.drop('popularity',1).values

<ipython-input-18-300b452bd692>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x=df.drop('popularity',1).values


In [19]:
popularity_counts = df['popularity'].value_counts()
popularity_counts

0      158391
1       39820
2       31247
8       30322
9       29249
        ...  
93          3
94          1
96          1
100         1
95          1
Name: popularity, Length: 98, dtype: int64

In [20]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace= list(popularity_counts[popularity_counts<30000].index)
# Replace in dataframe
for cls in classifications_to_replace:
    df['popularity'] = df['popularity'].replace(cls,"Other")
# Check to make sure binning was successful
df['popularity'].value_counts()

Other    899984
0        158391
1         39820
2         31247
8         30322
Name: popularity, dtype: int64

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [23]:
number_input_features = len(x_train[0])
hidden_nodes_layer1 = 128
hidden_nodes_layer2 = 64
hidden_nodes_layer3 = 32
nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

nn.add(tf.keras.layers.Dense(units=1, activation="linear"))
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])
# Training the model with Early Stopping

fit_model = nn.fit(
    x_train_scaled,
    y_train,
    epochs=100
)

Epoch 1/100
28995/28995 [==============================] - 94s 3ms/step - loss: 97.5162 - accuracy: 0.0371
Epoch 2/100
28995/28995 [==============================] - 74s 3ms/step - loss: 92.6602 - accuracy: 0.0378
Epoch 3/100
28995/28995 [==============================] - 73s 3ms/step - loss: 91.1617 - accuracy: 0.0370
Epoch 4/100
28995/28995 [==============================] - 73s 3ms/step - loss: 90.1278 - accuracy: 0.0365
Epoch 5/100
28995/28995 [==============================] - 73s 3ms/step - loss: 89.5026 - accuracy: 0.0375
Epoch 6/100
28995/28995 [==============================] - 73s 3ms/step - loss: 88.9563 - accuracy: 0.0377
Epoch 7/100
28995/28995 [==============================] - 72s 2ms/step - loss: 88.5851 - accuracy: 0.0379
Epoch 8/100
28995/28995 [==============================] - 71s 2ms/step - loss: 88.1324 - accuracy: 0.0382
Epoch 9/100
28995/28995 [==============================] - 72s 2ms/step - loss: 87.7943 - accuracy: 0.0384
Epoch 10/100
28995/28995 [===========

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

7249/7249 - 10s - loss: 85.4784 - accuracy: 0.0498 - 10s/epoch - 1ms/step
Loss: 85.47840118408203, Accuracy: 0.0497557707130909
